# type of parameter

In [6]:
import torch as tr
import numpy as np
class Net(tr.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.linears = tr.nn.ModuleList([tr.nn.Linear(3,4),tr.nn.Linear(4,6)])
        self.W = tr.nn.Parameter(tr.FloatTensor(np.random.randn(3,4)),requires_grad=True)
        self.W2 = tr.nn.Parameter(tr.FloatTensor(np.random.randn(3,4)),requires_grad=True)
        print('=========')
        print(self.linears[0])
        print('=========')
    
    def forward(self):
        X=tr.autograd.Variable(tr.FloatTensor(np.ones(shape=(2,3))))
        return self.W.matmul(X)

print('Paramter is a subclass of Variable. It can be registerd automatically in Module.')
module = Net()
t=module.forward()
print(t.data.numpy())
for name,para in list(module.named_parameters()):
    print(name)
    print(para)

Paramter is a subclass of Variable. It can be registerd automatically in Module.
Linear(in_features=3, out_features=4, bias=True)


RuntimeError: size mismatch, m1: [3 x 4], m2: [2 x 3] at /opt/conda/conda-bld/pytorch-cpu_1518282373170/work/torch/lib/TH/generic/THTensorMath.c:1434

In [32]:
import torch as tr
import numpy as np
class Net(tr.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.W = tr.nn.Parameter(tr.Tensor(np.ones((3,4))))
    
    def forward(self, X):
        z = tr.matmul(X,self.W)
        return z

module = Net()
X = tr.autograd.Variable(tr.Tensor(np.ones((3,3),dtype='float32')),requires_grad=False)
outputs = module(X)
print(outputs)
for name,para in list(module.named_parameters()):
    print(name)
    print(para)

Variable containing:
 3  3  3  3
 3  3  3  3
 3  3  3  3
[torch.FloatTensor of size 3x4]

W
Parameter containing:
 1  1  1  1
 1  1  1  1
 1  1  1  1
[torch.FloatTensor of size 3x4]



# Parameter will not be registered if it is created outside __init__()

In [33]:
import torch as tr
import numpy as np
class Net(tr.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
    
    def forward(self, X):
        W = tr.nn.Parameter(tr.Tensor(np.ones((3,4))))
        z = tr.matmul(X,W)
        return z

module = Net()
X = tr.autograd.Variable(tr.Tensor(np.ones((3,3),dtype='float32')),requires_grad=False)
outputs = module(X)
print(outputs)
for name,para in list(module.named_parameters()):
    print(name)
    print(para)

Variable containing:
 3  3  3  3
 3  3  3  3
 3  3  3  3
[torch.FloatTensor of size 3x4]



# Is forward function necessary?

In [47]:
import torch as tr
import torch.nn.functional as F
import numpy as np
class Net(tr.nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.linears = tr.nn.ModuleList([tr.nn.Linear(10,100),tr.nn.Linear(10,50)])
    
    def forward_softmax(self,X):
        linear_layer = self.linears[0](X)
        outputs = F.log_softmax(linear_layer,dim=1)
        return outputs
    
    def cross_entropy_loss(self,score,target):
        loss = tr.nn.NLLLoss(size_average=True,reduce=True)
        return loss(score,target)
        
    

# 
def optimizer(module):
    optim = tr.optim.SGD(module.parameters(),lr=0.03,weight_decay=0.0003)
    return optim
    
module = Net()
optim = optimizer(module)
X = tr.autograd.Variable(tr.Tensor(np.random.randn(30,10)),requires_grad=False)
y_ = tr.autograd.Variable(tr.LongTensor(np.random.randint(low=0,high=3,size=(30))))


for i in range(10):
    optim.zero_grad()
    score = module.forward_softmax(X)
    loss = module.cross_entropy_loss(score,y_)
    loss.backward()
    optim.step()
    for i,j in module.named_parameters():
        if i == 'linears.0.weight':
            print(i)
            print(j)


linears.0.weight
Parameter containing:
-0.2838  0.0697 -0.0463  ...  -0.1900 -0.1006 -0.0856
-0.1341  0.1609 -0.2762  ...   0.0939  0.1497 -0.0938
-0.2266  0.0573  0.2459  ...   0.0056 -0.1942  0.2193
          ...             ⋱             ...          
 0.2126  0.0881  0.1800  ...   0.0983 -0.1551 -0.2113
-0.2789 -0.2155  0.2749  ...   0.2006 -0.2932  0.0632
-0.2642 -0.1219  0.2691  ...  -0.2087 -0.0072  0.2242
[torch.FloatTensor of size 100x10]

linears.0.weight
Parameter containing:
-0.2858  0.0702 -0.0440  ...  -0.1807 -0.1035 -0.0842
-0.1248  0.1598 -0.2843  ...   0.1005  0.1557 -0.0893
-0.2326  0.0561  0.2445  ...   0.0003 -0.1931  0.2189
          ...             ⋱             ...          
 0.2125  0.0881  0.1801  ...   0.0981 -0.1551 -0.2113
-0.2787 -0.2155  0.2748  ...   0.2004 -0.2931  0.0631
-0.2641 -0.1218  0.2691  ...  -0.2088 -0.0072  0.2240
[torch.FloatTensor of size 100x10]

linears.0.weight
Parameter containing:
-0.2877  0.0708 -0.0417  ...  -0.1714 -0.1063 -0.0828
-